In [1]:
from kerchunk.netCDF3 import NetCDF3ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
import os
import ujson
from pathlib import Path
import xarray as xr
import numpy as np

In [2]:
dir = 'ne_pacific_10km/json'

In [3]:
fs = fsspec.filesystem('')
flist = fs.glob('iron*.nc')
flist
glist = fs.glob('/home/aclim/GOA files for Roland/roms*')
flist = flist + glist
flist

['/home/users/rhs/notebooks/GoA/iron_2020_01.nc',
 '/home/users/rhs/notebooks/GoA/iron_2020_02.nc',
 '/home/aclim/GOA files for Roland/roms_grd_nep.nc']

In [4]:
os.makedirs(dir, exist_ok=True)

In [5]:
fs2 = fsspec.filesystem("file")
so = dict(default_fill_cache=False, default_cache_type='first')
for u in flist:
    dachunks = NetCDF3ToZarr(u)
    parts = u.split('/') # seperate file path to create a unique name for each json 
    fstem = Path(u).stem 
    outf = f'{dir}/{fstem}.json'
    print(outf)
    with fs2.open(outf, 'wb') as f:
        f.write(ujson.dumps(dachunks.translate()).encode());

ne_pacific_10km/json/iron_2020_01.json
ne_pacific_10km/json/iron_2020_02.json
ne_pacific_10km/json/roms_grd_nep.json


In [6]:
json_list = fs2.glob(f'{dir}/iron*.json')
json_list

['/home/users/rhs/notebooks/GoA/ne_pacific_10km/json/iron_2020_01.json',
 '/home/users/rhs/notebooks/GoA/ne_pacific_10km/json/iron_2020_02.json']

In [7]:
%%time
mzz = MultiZarrToZarr(json_list,  
    concat_dims=['ocean_time'], #this is the dimension along which the individual files will be merged
    identical_dims = ['lat_rho', 'lon_rho', 's_rho', 's_w'],
)
d = mzz.translate()
with fs2.open(f'{dir}/combined.json', 'wb') as f:
    f.write(ujson.dumps(d).encode())

CPU times: user 7.6 ms, sys: 2.96 ms, total: 10.6 ms
Wall time: 9.05 ms
